In [ ]:
%matplotlib inline


Registration to template
============================
Registration of anatomical and functional scans to a given template.




Retrieve the fMRI data
----------------------



In [ ]:
from sammba import data_fetchers

retest = data_fetchers.fetch_zurich_test_retest(subjects=[0],
                                                correct_headers=True)

retest contains paths to images and data description



In [ ]:
anat_filename = retest.anat[0]
func_filename = retest.func[0]
print(func_filename)

We encapsulate them through an object called `FMRISession`



In [ ]:
from sammba.registration import FMRISession

animal_session = FMRISession(anat=anat_filename, func=func_filename,
                             animal_id='1366')

Choose the template
-------------------
We choose the downsampled version of DORR average, to avoid memory issues



In [ ]:
dorr = data_fetchers.fetch_atlas_dorr_2008(downsample='100')

dorr contains paths to DORR atlas and average T2 image



In [ ]:
template_filename = dorr.t2
print(template_filename)

Define the writing directory
----------------------------



In [ ]:
import os

write_dir = os.path.join(os.getcwd(), 'zurich_template')
if not os.path.exists(write_dir):
    os.makedirs(write_dir)

Perform the registration
------------------------
We purposely choose a low maxlev, to speed-up computations



In [ ]:
from sammba.registration import fmri_sessions_to_template

fmri_sessions_to_template(sessions=[animal_session],
                          head_template_filename=template_filename,
                          t_r=1., write_dir=write_dir,
                          brain_volume=400,
                          slice_timing=True,
                          prior_rigid_body_registration=True,
                          maxlev=1,
                          caching=True)

The paths to the registered images are added to `animal_session`



In [ ]:
print('Registrered anatomical image is ' + animal_session.registered_anat_)
print('Registrered functional image is ' + animal_session.registered_func_)

Visalize results
----------------
We plot the edges of the template on top of the registered anatomical image



In [ ]:
from nilearn import plotting, image

display = plotting.plot_anat(animal_session.registered_anat_, dim=-1.95,
                             title='template edges on top of registered anat')
display.add_edges(template_filename)

and on top of the average registered functional



In [ ]:
from nilearn import image

mean_registered_func_filename = image.mean_img(animal_session.registered_func_)
display = plotting.plot_epi(mean_registered_func_filename,
                            title='template edges on top of registered func')
display.add_edges(template_filename)
plotting.show()